In [153]:
import sympy as sp
import numpy as np 
from numpy import radians
from time import perf_counter

In [154]:
num_poles = 3

pole_as = [0.05+0.05*i for i in range(num_poles)]
pole_ls = [0.1+0.1*i for i in range(num_poles)]
pole_ms = [0.1+0.05*i for i in range(num_poles)]
pole_ds = [0.01 for i in range(num_poles)]
pole_Js = [0.13-0.01*i for i in range(num_poles)]

m_c = 0.5
g = 9.81

In [155]:
s = sp.Symbol("s")
d_s = sp.Symbol("\\dot{s}")
dd_s = sp.Symbol("\\ddot{s}")
thetas = [sp.Symbol(f"\\theta_{i+1}") for i in range(num_poles)]
d_thetas = [sp.Symbol(f"\\dot{{\\theta}}_{i+1}") for i in range(num_poles)]
dd_thetas = [sp.Symbol(f"\\ddot{{\\theta}}_{i+1}") for i in range(num_poles)]
u = dd_s

derivate_by_time_dict = {s: d_s, d_s: dd_s}
for i in range(num_poles):
    derivate_by_time_dict[thetas[i]] = d_thetas[i]
    derivate_by_time_dict[d_thetas[i]] = dd_thetas[i]

In [156]:
pole_pcs = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*sp.sin(prev_theta)
        prev_2 += prev_l*sp.cos(prev_theta)
    pole_pcs.append([s-a*sp.sin(theta)+prev_1, a*sp.cos(theta)+prev_2])
print(pole_pcs)

[[s - 0.05*sin(\theta_1), 0.05*cos(\theta_1)], [s - 0.1*sin(\theta_1) - 0.1*sin(\theta_2), 0.1*cos(\theta_1) + 0.1*cos(\theta_2)], [s - 0.1*sin(\theta_1) - 0.2*sin(\theta_2) - 0.15*sin(\theta_3), 0.1*cos(\theta_1) + 0.2*cos(\theta_2) + 0.15*cos(\theta_3)]]


In [161]:
def square_sum(x: tuple):
    result = 0
    for x_i in x:
        result += x_i**2
    print(result)
    return result

T = [1/2*m_c*d_s**2]
for m, pc, J, d_theta in zip(pole_ms, pole_pcs, pole_Js, d_thetas):
    T_p = 1/2*m*square_sum(pc) + 1/2*J*d_theta**2
    T.append(T_p)
    
V = 0   
for m, pc in zip(pole_ms, pole_pcs):
    V += g*m*pc[1]
R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2
    prev_w = d_theta

(s - 0.05*sin(\theta_1))**2 + 0.0025*cos(\theta_1)**2
0.01*(cos(\theta_1) + cos(\theta_2))**2 + (s - 0.1*sin(\theta_1) - 0.1*sin(\theta_2))**2
0.04*(0.5*cos(\theta_1) + cos(\theta_2) + 0.75*cos(\theta_3))**2 + (s - 0.1*sin(\theta_1) - 0.2*sin(\theta_2) - 0.15*sin(\theta_3))**2


In [162]:
eqs = []
# lh = sp.diff(L, d_s).subs(derivate_by_time_dict)
# lh -= sp.diff(L, s) + sp.diff(R, d_s)
# rh = u
# eqs.append(lh-rh)
for theta, d_theta, T_p in zip(thetas, d_thetas, T[1:]):
    L = T_p-V
    lh = sp.diff(L, d_theta).subs(derivate_by_time_dict)
    lh -= sp.diff(L, theta) + sp.diff(R, d_theta)
    rh = 0
    eqs.append(lh-rh)
eqs

[0.13*\ddot{\theta}_1 - 0.02*\dot{\theta}_1 + 0.01*\dot{\theta}_2 + 0.005*(s - 0.05*sin(\theta_1))*cos(\theta_1) + 0.00025*sin(\theta_1)*cos(\theta_1) - 0.3924*sin(\theta_1),
 0.12*\ddot{\theta}_2 + 0.01*\dot{\theta}_1 - 0.02*\dot{\theta}_2 + 0.01*\dot{\theta}_3 + 0.0015*(cos(\theta_1) + cos(\theta_2))*sin(\theta_2) + 0.015*(s - 0.1*sin(\theta_1) - 0.1*sin(\theta_2))*cos(\theta_2) - 0.53955*sin(\theta_2),
 0.11*\ddot{\theta}_3 + 0.01*\dot{\theta}_2 - 0.01*\dot{\theta}_3 + 0.006*(0.5*cos(\theta_1) + cos(\theta_2) + 0.75*cos(\theta_3))*sin(\theta_3) + 0.03*(s - 0.1*sin(\theta_1) - 0.2*sin(\theta_2) - 0.15*sin(\theta_3))*cos(\theta_3) - 0.2943*sin(\theta_3)]

In [163]:
sol = sp.solve(eqs, dd_thetas)
sol = {k: v.simplify() for k, v in sol.items()}

In [169]:
values = np.array([0, 0, radians(1), 0, radians(1), 0, radians(1), 0] + [0])
variables = [s, d_s]
for theta, d_theta in zip(thetas, d_thetas):
    variables.append(theta)
    variables.append(d_theta)
variables.append(u)

start_time = perf_counter()
# substitute the values of the variables into the solutions
sol_sub = sol.copy()
for i in range(1):
    for k, v in sol.items():
        sol_sub[k] = v.subs({k: v for k, v in zip(variables, values)})
        print(f"{k}: {sol_sub[k]}")
print(f"Time: {perf_counter()-start_time}")

\ddot{\theta}_1: 0.0526794175845388
\ddot{\theta}_2: 0.000218155080466044
\ddot{\theta}_3: 0.000475974721016823
Time: 0.03662220004480332
